## Grupo

227124 Jorge Henrique Monteiro dos Santos

262251 Vitor Mello de Araujo Lima

## Objetivo

Aplicar os vários regressores visto em aula. Busca de hiperparametros.


In [14]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame as df
from statistics import mean
import math
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, TimeSeriesSplit, GridSearchCV,RandomizedSearchCV
import numpy as np
from sklearn.model_selection import StratifiedKFold

# Leia

## Leia o arquivo dados4.csv.


In [3]:
url = "https://www.ic.unicamp.br/~wainer/cursos/1s2021/432/dados4.csv"
data = pd.read_csv(url)
data = data[::-1]
data.shape

(690, 15)

In [4]:
data[::]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
689,1,41.00,0.040,2,e,bb,0.040,0,1,1,0,s,560,1,1
688,0,27.42,14.500,2,x,h,3.085,1,1,1,0,g,120,12,1
687,0,18.83,9.540,2,aa,bb,0.085,1,0,0,0,g,100,1,1
686,1,20.67,0.415,2,c,bb,0.125,0,0,0,0,g,0,45,0
685,1,31.57,10.500,2,x,bb,6.500,1,0,0,0,g,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,1,20.17,8.170,2,aa,bb,1.960,1,1,14,0,g,60,159,1
3,0,21.67,11.500,1,j,j,0.000,1,1,11,1,g,0,1,1
2,0,29.58,1.750,1,k,bb,1.250,0,0,0,1,g,280,1,0
1,0,22.67,7.000,2,c,bb,0.165,0,0,0,0,g,160,1,0


##  Converta os atributos categóricos para numéricos (V5,V6,V12)

In [5]:
V5 = pd.get_dummies(data['V5'], prefix='V5')
V6 = pd.get_dummies(data['V6'], prefix='V6')
V12 = pd.get_dummies(data['V12'], prefix='V12')

data_num = data.drop(['V5', 'V6','V12' ],axis=1)
data_num = pd.concat([data_num,V5,V6,V12], axis=1)

data_num.head()

,V1,V2,V3,V4,V7,V8,V9,V10,V11,V13,...,V6_dd,V6_ff,V6_h,V6_j,V6_o,V6_v,V6_z,V12_g,V12_p,V12_s
689,1,41.00,0.040,2,0.040,0,1,1,0,560,...,0,0,0,0,0,0,0,0,0,1
688,0,27.42,14.500,2,3.085,1,1,1,0,120,...,0,0,1,0,0,0,0,1,0,0
687,0,18.83,9.540,2,0.085,1,0,0,0,100,...,0,0,0,0,0,0,0,1,0,0
686,1,20.67,0.415,2,0.125,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
685,1,31.57,10.500,2,6.500,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
y_data = data_num['V15']
x_data = data_num.drop(['V15'], axis=1)

,V1,V2,V3,V4,V7,V8,V9,V10,V11,V13,...,V6_dd,V6_ff,V6_h,V6_j,V6_o,V6_v,V6_z,V12_g,V12_p,V12_s
689,1,41.00,0.040,2,0.040,0,1,1,0,560,...,0,0,0,0,0,0,0,0,0,1
688,0,27.42,14.500,2,3.085,1,1,1,0,120,...,0,0,1,0,0,0,0,1,0,0
687,0,18.83,9.540,2,0.085,1,0,0,0,100,...,0,0,0,0,0,0,0,1,0,0
686,1,20.67,0.415,2,0.125,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
685,1,31.57,10.500,2,6.500,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# Centering and Scaling

In [10]:
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame

x_data_scaled = DataFrame(StandardScaler().fit_transform(x_data), columns=x_data.columns)
x_data_scaled.head()

,V1,V2,V3,V4,V7,V8,V9,V10,V11,V13,...,V6_dd,V6_ff,V6_h,V6_j,V6_o,V6_v,V6_z,V12_g,V12_p,V12_s
0,0.688737,0.796290,-0.948572,0.54295,-0.652915,-1.047504,1.157144,-0.288101,-0.919195,2.185524,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,-3.100868,-0.108306,3.332456
1,-1.451933,-0.350217,1.958221,0.54295,0.257647,0.954650,1.157144,-0.288101,-0.919195,-0.372103,...,-0.093659,-0.300079,2.0,-0.108306,-0.093659,-0.305782,-0.108306,0.322490,-0.108306,-0.300079
2,-1.451933,-1.075437,0.961146,0.54295,-0.639459,0.954650,-0.864196,-0.493887,-0.919195,-0.488358,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.322490,-0.108306,-0.300079
3,0.688737,-0.920093,-0.873189,0.54295,-0.627497,-1.047504,-0.864196,-0.493887,-0.919195,-1.069637,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.322490,-0.108306,-0.300079
4,0.688737,0.000152,1.154129,0.54295,1.278852,0.954650,-0.864196,-0.493887,-0.919195,-1.069637,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.322490,-0.108306,-0.300079


# Classificação 

In [30]:
# Generate all classifiers and parameters
import pprint
from sklearn.linear_model import LogisticRegression
import scipy.stats as stats
from sklearn.utils.fixes import loguniform
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
import random
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import uniform
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB


test_model = {

    'Logistic Regression': {
        'regressor' : LogisticRegression,
        'parameters': {'max_iter': [2000]},
        'n_iter'    : 1,
        'results'   : {}
    },
    'Logistic Regression - L2': {
        'regressor': LogisticRegression,
        'parameters': {
            'penalty': ['l2'],
            'C' : loguniform(10**-3, 10**3),
            'max_iter': [2000]
        },
        'n_iter' : 10,
        'results' : {}
    },
    'Linear Discriminant Analysisr':{
        'regressor': LinearDiscriminantAnalysis,
        'parameters': {},
        'n_iter'    : 1,
        'results'   : {}
    },
    'Quadratic Discriminant Analysis':{
        'regressor': QuadraticDiscriminantAnalysis,
        'parameters': {},
        'n_iter'    : 1,
        'results'   : {}
    },
    'K-Nearest Neighbors': {
        'regressor': KNeighborsClassifier,
        'parameters': {
            'n_neighbors': stats.randint(1,302)
            },
        'n_iter' : 10,
        'results' : {} 
    },
    #'SVM Linear': {
    #    'regressor': SVC,
    #    'parameters': {
    #        'C'      : loguniform(2**-5, 2**15),
    #        'kernel' : ['linear']
    #    },
    #    'n_iter' : 10,
    #    'results' : {} 
    #},
    'SVM RBF': {
        'regressor': SVC,
        'parameters': {
            'C'      : loguniform(2**-5, 2**15),
            'gamma'  : loguniform(2**-9, 2**3),
            'kernel' : ['rbf'],
            'probability': [True]
        },
        'n_iter' : 10,
        'results' : {} 
    },
    'Multi-layer Perceptron': {
        'regressor': MLPClassifier,
        'parameters': {
            'hidden_layer_sizes': np.arange(5, 20+1, 3),
            'max_iter': [5000]
            },
        'n_iter' : 10,
        'results' : {} 
    },
    'Decision Tree': {
        'regressor': DecisionTreeClassifier,
        'parameters': {
            'ccp_alpha': uniform(loc=0.0, scale=0.04)
            },
        'n_iter' : 10,
        'results' : {} 
    },
    'Random Forest': {
        'regressor': RandomForestClassifier,
        'parameters': {
            'n_estimators': [10, 100, 1000],
            'max_features': [5,8,10]
            },
        'n_iter' : 10,
        'results' : {} 
    },
    'Gradient Boost Machine': {
        'regressor': GradientBoostingClassifier,
        'parameters': {
            'n_estimators': np.arange(5, 101, 1),
            'learning_rate': uniform(loc=0.01, scale=(0.3-0.01)),
            'max_depth': [2,3]
            },
        'n_iter' : 10,
        'results' : {} 
    },
    'Gaussian Naive Bayes': {
        'regressor': GaussianNB,
        'parameters': {},
        'n_iter'    : 1,
        'results'   : {} 
    }
}

### Nested-Cross Validation

- Outer-loop: 4 repetições de split de 70% para treino e 30 para teste no outer loop
- Inner-loop: 3-fold estratificado

A implemetação utilizada nesse trabalho é baseada no seguinte exemplo:

https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from numpy import mean
from numpy import std

def nested_cross_val_random(model,parameters,X_data, y_data, n_it):
    auc_list = []
    for i in range(4):
        # Create test and train sets
        X_train,X_test,y_train,y_test=train_test_split(X_data, y_data, test_size=0.3)

        inner_cv = StratifiedKFold(n_splits=3)

        rnd_search = RandomizedSearchCV(estimator=model(), param_distributions=parameters, cv=inner_cv,scoring='roc_auc', refit=True, n_iter=n_it)
        result = rnd_search.fit(X_train, y_train)
        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_
        auc = roc_auc_score(y_test,best_model.predict_proba(X_test)[:, 1])
        auc_list.append(auc)
                        
    return mean(auc_list), std(auc_list)

    
for classifier in test_model.keys():
    print(classifier)
    auc_mean,auc_std = nested_cross_val_random(test_model[classifier]['regressor'],test_model[classifier]['parameters'],x_data_scaled,y_data, test_model[classifier]['n_iter'])
    test_model[classifier]['results']['auc_mean'] = auc_mean
    test_model[classifier]['results']['auc_std'] = auc_std

Logistic Regression
Logistic Regression - L2
Linear Discriminant Analysisr
Quadratic Discriminant Analysis


/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variab

K-Nearest Neighbors
SVM RBF
Multi-layer Perceptron


/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Decision Tree
Random Forest


/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/home/jorgehs/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Gradient Boost Machine
Gaussian Naive Bayes


In [48]:
cls_results_dict = {}

for classifier in test_model.keys():
    cls_results_dict[classifier] = {'auc_men':test_model[classifier]['results']['auc_mean'],
                                    'auc_std':test_model[classifier]['results']['auc_std']}

cls_results_dict



{'Logistic Regression': {'auc_men': 0.9336572576295867,
  'auc_std': 0.014751015397566614},
 'Logistic Regression - L2': {'auc_men': 0.9154834924458075,
  'auc_std': 0.012267024985725062},
 'Linear Discriminant Analysisr': {'auc_men': 0.924771110268171,
  'auc_std': 0.015746007422063134},
 'Quadratic Discriminant Analysis': {'auc_men': 0.786152354966595,
  'auc_std': 0.05588041588377054},
 'K-Nearest Neighbors': {'auc_men': 0.921039138345511,
  'auc_std': 0.009394440021460906},
 'SVM RBF': {'auc_men': 0.9074268455988208, 'auc_std': 0.018355925517217025},
 'Multi-layer Perceptron': {'auc_men': 0.8906208076744179,
  'auc_std': 0.036522679034184734},
 'Decision Tree': {'auc_men': 0.9038186189603749,
  'auc_std': 0.008772669112649768},
 'Random Forest': {'auc_men': 0.9288875606008482,
  'auc_std': 0.009278284729253461},
 'Gradient Boost Machine': {'auc_men': 0.9408265107431356,
  'auc_std': 0.022322294724797194},
 'Gaussian Naive Bayes': {'auc_men': 0.8333895993699888,
  'auc_std': 0.02807

In [49]:
cls_table = df.from_dict(cls_results_dict, orient='index')
cls_table

,auc_men,auc_std
Logistic Regression,0.933657,0.014751
Logistic Regression - L2,0.915483,0.012267
Linear Discriminant Analysisr,0.924771,0.015746
Quadratic Discriminant Analysis,0.786152,0.055880
K-Nearest Neighbors,0.921039,0.009394
SVM RBF,0.907427,0.018356
Multi-layer Perceptron,0.890621,0.036523
Decision Tree,0.903819,0.008773
Random Forest,0.928888,0.009278
Gradient Boost Machine,0.940827,0.022322
